### 검증 세트
테스트 세트를 사용하지 않고 훈련 세트를 또 나누어 사용

In [3]:
import pandas as pd
wine = pd.read_csv('https://bit.ly/wine_csv_data')

In [4]:
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

In [5]:
from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)

In [6]:
sub_input, val_input, sub_target, val_target = train_test_split(train_input, train_target, test_size=0.2, random_state=42)

훈련 세트와 검증 세트의 크기 확인해보기

In [7]:
print(sub_input.shape, val_input.shape)

(4157, 3) (1040, 3)


모델 훈련해보기
- 결과 값: 과대적합

In [8]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)
print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

0.9971133028626413
0.864423076923077


## 교차 검증(Cross validation)
1. 검증 세트를 떼어 내어 평가하는 과정을 여러번 반복한다.
2. 점수를 평균하여 최종 검증 점수를 얻는다.
- 3-폴드 교차 검증: 훈련 세트를 세 부분으로 나눠서 교차 검증을 수행하는 것(=k-겹 교차 검증)

In [9]:
from sklearn.model_selection import cross_validate
scores = cross_validate(dt, train_input, train_target)
print(scores)

{'fit_time': array([0.00399232, 0.0034678 , 0.0036521 , 0.00348973, 0.00333786]), 'score_time': array([0.00048423, 0.0003767 , 0.00042772, 0.0003643 , 0.0004158 ]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


교차 검증의 최종 점수

In [10]:
import numpy as np
print(np.mean(scores['test_score']))

0.855300214703487


훈련 세트를 섞은 후 10-폴드 교차 검증 수행하기

In [11]:
from sklearn.model_selection import StratifiedKFold
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_validate(dt, train_input, train_target, cv=splitter)
print(np.mean(scores['test_score']))

0.8574181117533719


### 하이퍼파라미터 튜닝
사용자 지정 파라미터

- 0.0001 부터 0.0005 까지 증가하는 5개의 값을 시도
- GridSearchCV : 하이퍼파라미터 탐색, 교차검증 수행
    - cv 매개변수 기본 값: 5 -> 5폴드교차검증 -> 25개 모델 훈련
    - n_jobs 매개변수 기본 값: 1 , CPU 코어의 수 결정 (-1: 모든 코어 사용)

In [12]:
from sklearn.model_selection import GridSearchCV
params = {'min_impurity_decrease': [0.0001, 0.0002, 0.0003, 0.0004, 0.005]}

In [13]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42,), params, n_jobs=-1)

In [14]:
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.005]})

훈련 후 검증 점수가 가장 높은 모델의 매개변수 조합으로 재훈련
- best_estimator_ 속성에 저장

In [15]:
dt = gs.best_estimator_
print(dt.score(train_input, train_target))

0.9615162593804117


그리드 서치로 찾은 최적의 매개변수
- best_params_ 속성에 저장

In [16]:
print(gs.best_params_)

{'min_impurity_decrease': 0.0001}


각 매개변수에서 수행한 교차 검증의 평균 점수
- 결과 값: 첫번째 값이 가장 크다

In [17]:
print(gs.cv_results_['mean_test_score'])

[0.86819297 0.86453617 0.86492226 0.86780891 0.85318557]


In [19]:
best_index = np.argmax(gs.cv_results_['mean_test_score'])
print(gs.cv_results_['params'][best_index]) # 함수 사용

{'min_impurity_decrease': 0.0001}


- 9 x 15 x 10 = 1,350 : 교차 검증 횟수
- 기본인 5-폴드교차검증 수행 -> 모델의 수: 6,750 개

In [20]:
params = {'min_impurity_decrease': np.arange(0.0001, 0.001, 0.0001),
          'max_depth': range(5,20,1),
          'min_samples_split': range(2, 100, 10)}

In [21]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 10)})

In [22]:
print(gs.best_params_)

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}


In [26]:
print(np.argmax(gs.cv_results_['mean_test_score']))

841


### 랜덤 서치
- 매개변수 값의 목록이 아닌 샘플링할 수 있는 확률 분포 객체를 전달한다.

In [28]:
from scipy.stats import uniform, randint # 균등 분포에서 샘플링, randint: 정수 / uniform: 실수

In [32]:
rgen = randint(0,10)
rgen.rvs(10)

array([8, 7, 2, 6, 1, 9, 1, 4, 9, 2])

In [33]:
np.unique(rgen.rvs(1000), return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([ 96,  97, 105, 108,  98, 105,  97, 106, 105,  83]))

In [34]:
ugen = uniform(0, 1)
ugen.rvs(10)

array([0.72724093, 0.34671813, 0.90059045, 0.7355337 , 0.41615186,
       0.87771539, 0.9128475 , 0.13165003, 0.51885899, 0.42136768])

In [35]:
params = {'min_impurity_decrease': uniform(0.0001, 0.001),
          'max_depth' : randint(20, 50),
          'min_samples_split': randint(2,25),
          'min_samples_leaf': randint(1, 25),}

In [42]:
from sklearn.model_selection import RandomizedSearchCV
gs= RandomizedSearchCV(DecisionTreeClassifier(random_state=42, splitter='random'), params, 
                       n_iter=100, n_jobs=-1, random_state=42)
gs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42,
                                                    splitter='random'),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7f184ea139e0>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7f184c948dd0>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7f184ce106e0>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7f184c948fe0>},
                   random_state=42)

In [43]:
print(gs.best_params_)

{'max_depth': 43, 'min_impurity_decrease': 0.00011407982271508446, 'min_samples_leaf': 19, 'min_samples_split': 18}


In [44]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8458726956392981


테스트 점수는 검증 점수보다 조금 작은 것이 일반적이다.

In [45]:
dt = gs.best_estimator_
print(dt.score(test_input, test_target))

0.786923076923077
